# Import data

In [17]:
import pandas as pd

In [18]:
# import data
dir = '../data/raw/'
filename = 'BMMS_overview.xlsx'
df_bridges = pd.read_excel(dir + filename)

In [19]:
# import roads_transposed.csv
dir = '../data/processed/'
filename = '_roads_cleaned.csv' # replace this with the interpolated data
df_roads = pd.read_csv(dir + filename)

# Step 1: Clean bridge data

In [20]:
# Swap 'lat' and 'lon' if both are out of bounds

# Define latitude and longitude boundaries
LAT_MIN, LAT_MAX = 20, 28
LON_MIN, LON_MAX = 88, 93

def swap_coordinates(row):
    if not (LAT_MIN <= row['lat'] <= LAT_MAX) and not (LON_MIN <= row['lon'] <= LON_MAX):
        row['lat'], row['lon'] = row['lon'], row['lat']  # Swap values
    return row

def clean_bridge_data(df_bridges):
    # make a copy of the original data
    bridge = df_bridges.copy()

    # delete the rows with missing values in column 'lat' or 'lon'
    bridge_new = bridge.dropna(subset=['lat', 'lon'])

    # delete the rows with 0 in column 'lat' or 'lon'
    bridge_new = bridge_new[(bridge_new['lat'] != 0) & (bridge_new['lon'] != 0)]

    # create a new column 'road_LRPName' by combining 'road' and 'LRPName'
    bridge_new = bridge_new.copy()
    bridge_new['road_LRPName'] = bridge_new['road'] + '_' + bridge_new['LRPName']

    # add a column 'null_num' with the calculattion of the number of null values for the each row
    bridge_new['null_num'] = bridge_new.isnull().sum(axis=1)

    # delete the rows with duplicated 'road_LRPName' and keep the row with the minimum number of null values
    bridge_new = bridge_new.loc[bridge_new.groupby('road_LRPName')['null_num'].idxmin()]

    # find the duplicated 'road_LRPName'
    bridge_new[bridge_new.duplicated(subset='road_LRPName', keep=False)]

    # sort the data by 'road'
    #bridge_new = bridge_new.sort_values(by=['road', 'LRPName'])

    # Find the rows with 'lat' and 'lon' out of bounds
    bridge_new[(bridge_new['lat'] < LAT_MIN) |
               (bridge_new['lat'] > LAT_MAX) |
               (bridge_new['lon'] < LON_MIN) | 
               (bridge_new['lon'] > LON_MAX)]
    
    bridge_new = bridge_new.apply(swap_coordinates, axis=1)

    # Find the rows with 'lat' and 'lon' out of bounds
    bridge_new[(bridge_new['lat'] < LAT_MIN) | 
               (bridge_new['lat'] > LAT_MAX) | 
               (bridge_new['lon'] < LON_MIN) | 
               (bridge_new['lon'] > LON_MAX)]
    
    return bridge_new

cleaned_bridges = clean_bridge_data(df_bridges)
#cleaned_bridges

## Save excel file

In [21]:
# Save the cleaned data to a .xlsx file
dir = '../data/processed/'
filename = 'BMMS_overview_cleaned_prelim.xlsx'
cleaned_bridges.to_excel(dir + filename, index=False, sheet_name='BMMS_overview')

# Step 2: Remove bridges without roads

In [22]:
# Create a dictionary to store the LRPE for each road
lrpe_dict = {}

# Create a dictionary to store the one LRP before LRPE for each road
lrp_before_lrpe = {}

# Iterate over each unique road
for road in df_roads['road'].unique():
    # Get the rows for the current road
    road_rows = df_roads[df_roads['road'] == road]
    
    # Find the index of LRPE
    if 'LRPE' in road_rows['lrp'].values:
        lrpe_indices = road_rows[road_rows['lrp'] == 'LRPE'].index
        if len(lrpe_indices) > 0:
            lrpe_index = lrpe_indices[0]
            lrpe_dict[road] = road_rows.loc[lrpe_index, 'lrp']
            if lrpe_index > 0 and lrpe_index - 1 < len(road_rows):
                lrp_before_lrpe[road] = road_rows.iloc[lrpe_index - 1]['lrp']
            else:
                lrp_before_lrpe[road] = road_rows.iloc[-2]['lrp']
        else:
            lrp_before_lrpe[road] = road_rows.iloc[-2]['lrp']
    else:
        # Remember the last LRP if there is no LRPE
        lrp_before_lrpe[road] = road_rows.iloc[-1]['lrp']
        lrpe_dict[road] = None  # Add a placeholder for LRPE

# Combine the dictionaries into a dataframe
df_lrpe_road = pd.DataFrame({
    'road': lrpe_dict.keys(),
    'LRP_before_E': lrp_before_lrpe.values(),
    'LRPE': lrpe_dict.values()
})
#df_lrpe_road.head()

# Sort the dataframe by road and LRP, but keep 'LRPS' at the start
def custom_sort(df):
    df['lrp_order'] = df['LRPName'].apply(lambda x: 0 if 'LRPS' in x else 1)
    df = df.sort_values(by=['road', 'lrp_order', 'LRPName'])
    df = df.drop(columns=['lrp_order'])
    return df

cleaned_bridges_sorted = custom_sort(cleaned_bridges)

import re

# Function to extract numeric part from LRPName
def extract_numeric(lrp_name):
    match = re.search(r'\d+', lrp_name)
    return int(match.group()) if match else float('inf')

# Function to remove rows with lrp after 'LRP_before_E' for each unique road
def remove_rows_after_lrp_before_e(df, df_lrpe):
    modified_rows = []
    for _, row in df_lrpe.iterrows():
        road = row['road']
        lrp_before_e = row['LRP_before_E']
        
        # Get the rows for the current road
        road_rows = df[df['road'] == road].copy()
        
        # Extract numeric part from LRPName and sort
        road_rows['lrp_numeric'] = road_rows['LRPName'].apply(extract_numeric)
        road_rows = road_rows.sort_values(by='lrp_numeric')
        
        # Get the index of the row with 'LRP_before_E'
        lrp_before_e_indices = road_rows[road_rows['LRPName'] == lrp_before_e].index
        
        if not lrp_before_e_indices.empty:
            lrp_before_e_index = lrp_before_e_indices[-1]  # Get the last occurrence
            # Drop rows after 'LRP_before_E'
            road_rows = road_rows[road_rows['lrp_numeric'] <= road_rows.loc[lrp_before_e_index, 'lrp_numeric']]
        
        modified_rows.append(road_rows)
    
    # Concatenate all modified rows
    df_modified = pd.concat(modified_rows)
    
    return df_modified.drop(columns=['lrp_numeric'])

# Apply the function to df_roads_bridges_combined_sorted
df_bridges_no_lrp_after_e = remove_rows_after_lrp_before_e(cleaned_bridges_sorted, df_lrpe_road)
#df_bridges_no_lrp_after_e

# Remove columns 'road_LRPName' and 'null_num'
df_final_bridges = df_bridges_no_lrp_after_e.drop(columns=['road_LRPName', 'null_num'])

# reset the index
df_final_bridges = df_final_bridges.reset_index(drop=True)

import numpy as np

# Create an empty dataframe to store the removed rows
removed_rows_df = pd.DataFrame(columns=cleaned_bridges.columns)

# Compare cleaned_bridges with df_final_bridges; save lrps that are removed
for road in df_final_bridges['road'].unique():
    cleaned_lrps = cleaned_bridges[cleaned_bridges['road'] == road]['LRPName'].values
    final_lrps = df_final_bridges[df_final_bridges['road'] == road]['LRPName'].values
    
    removed_lrps = np.setdiff1d(cleaned_lrps, final_lrps)
    
    if len(removed_lrps) > 0:
        removed_rows = cleaned_bridges[(cleaned_bridges['road'] == road) & (cleaned_bridges['LRPName'].isin(removed_lrps))]
        removed_rows_df = pd.concat([removed_rows_df, removed_rows])

removed_rows_df.reset_index(drop=True, inplace=True)
#removed_rows_df

C:\Users\Move\AppData\Local\Temp\ipykernel_23272\4205491286.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_modified = pd.concat(modified_rows)
C:\Users\Move\AppData\Local\Temp\ipykernel_23272\4205491286.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  removed_rows_df = pd.concat([removed_rows_df, removed_rows])


## Save excel file

In [23]:
# Save the cleaned data to a new excel file
dir = '../data/processed/'
filename = 'BMMS_overview_cleaned_bridges_after_LPRE_removed.xlsx'
df_final_bridges.to_excel(dir + filename, index=False, sheet_name='BMMS_overview')

In [24]:
# Save the cleaned data to a new excel file
dir = '../data/processed/'
filename = 'BMMS_overview_removed_lrps.xlsx'
removed_rows_df.to_excel(dir + filename, index=False, sheet_name='BMMS_overview')

# Step 3: Adjust Bridge Coordinates

In [25]:
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial import cKDTree

def adjust_bridge_coordinates_simple(df_bridges, df_roads):
    adjusted_bridges = df_bridges.copy()
    
    for road in df_bridges['road'].unique():
        road_rows = df_roads[df_roads['road'] == road]
        road_points = list(zip(road_rows['lon'], road_rows['lat']))
        if len(road_points) == 0:
            continue
        tree = cKDTree(road_points)
        
        for idx, bridge_row in df_bridges[df_bridges['road'] == road].iterrows():
            bridge_point = (bridge_row['lon'], bridge_row['lat'])
            dist, idx_closest = tree.query(bridge_point)
            closest_point = road_points[idx_closest]
            
            adjusted_bridges.at[idx, 'lat'] = closest_point[1]
            adjusted_bridges.at[idx, 'lon'] = closest_point[0]
    
    return adjusted_bridges

df_final_bridges_adjusted = adjust_bridge_coordinates_simple(df_final_bridges, df_roads)
#df_final_bridges_adjusted


## Save excel file: Final

In [26]:
# Save the cleaned data to a .xlsx file
dir = '../data/processed/'
filename = 'BMMS_overview.xlsx'
df_final_bridges_adjusted.to_excel(dir + filename, index=False, sheet_name='BMMS_overview')